# Модуль weakref

Модуль weakref обеспечивает поддержку слабых ссылок на объекты. Особенностью слабой ссылки является то, что она позволяет ссылаться на объект, не препятствуя его автоматическому
удалению.

## weakref.ref

Со слабыми ссылками работают через класс ref. Чтобы обратиться к исходному объекту, следует вызвать объект ссылки.



In [ ]:
import weakref


class ExpensiveObject:

    def __del__(self):
        print('(Deleting {})'.format(self))


obj = ExpensiveObject()
r = weakref.ref(obj)

print('obj:', obj)
print('ref:', r)
print('r():', r())

print('deleting obj')
del obj
print('r():', r())

В данном случае второй вызов объекта ссылки, r (), возвращает значение
None, поскольку к этому времени исходный объект obj уже был удален.

## Функции обратного вызова в слабых ссылках

Конструктор ref поддерживает необязательный аргумент в виде функции обратного вызова, которая будет автоматически вызываться при удалении исходного объекта.

In [ ]:
import weakref


class ExpensiveObject:

    def __del__(self):
        print('(Deleting {})'.format(self))


def callback(reference):
    """Вызывается при удалении целевого объекта"""
    print('callback({!r})'.format(reference))


obj = ExpensiveObject()
r = weakref.ref(obj, callback)

print('obj:', obj)
print('ref:', r)
print('r():', r())

print('deleting obj')
del obj
print('r():', r())

Функция обратного вызова получает объект ссылки в качестве аргумента, когда ссылка становится недействительной по причине удаления объекта. Одним из возможных применений этого средства является удаление объекта слабой ссылки из кеша.

## weakref.finalize

Надежность управления ресурсами при удалении слабых ссылок можно повысить, связывая c объектами функции обратного вызова при помощи функции finalize(). Экземпляр finalize (объект-финализатор) удерживается в памяти до тех пор, пока не будет удален связанный c ним объект, даже если в приложении отсутствуют ссылки на объект-финализатор.

In [ ]:
import weakref


class ExpensiveObject:

    def __del__(self):
        print('(Deleting {})'.format(self))


def on_finalize(*args):
    print('on_finalize({!r})'.format(args))


obj = ExpensiveObject()
weakref.finalize(obj, on_finalize, 'extra argument')

del obj

Аргументами функции finalize() являются отслеживаемый объект, вызываемый объект, подлежащий вызову при удалении объекта сборщиком мусора, и любые позиционные или именованные аргументы, передаваемые вызываемому объекту.

## atexit

Экземпляр finalize имеет перезаписываемое свойство atexit, позволяющее
управлять вызовом функции обратного вызова при выходе из программы, если
она до этого не вызывалась.

In [2]:
import sys
import weakref


class ExpensiveObject:

    def __del__(self):
        print('(Deleting {})'.format(self))


def on_finalize(*args):
    print('on_finalize({!r})'.format(args))


obj = ExpensiveObject()
f = weakref.finalize(obj, on_finalize, 'extra argument')
f.atexit = bool(int(sys.argv[1]))

(Deleting <__main__.ExpensiveObject object at 0x000002BE18C45280>)
on_finalize(('extra argument',))


ValueError: invalid literal for int() with base 10: '-f'

Предоставление экземпляру finalize ссылки на объект приводит к тому, что
она удерживается в памяти, поэтому сборщик мусора не удалитданный объект.

In [ ]:
import gc
import weakref


class ExpensiveObject:

    def __del__(self):
        print('(Deleting {})'.format(self))


def on_finalize(*args):
    print('on_finalize({!r})'.format(args))


obj = ExpensiveObject()
obj_id = id(obj)

f = weakref.finalize(obj, on_finalize, obj)
f.atexit = False

del obj

for o in gc.get_objects():
    if id(o) == obj_id:
        print('found uncollected object in gc')

Как следует из этого примера, объект obj сохраняется в памяти даже после удаления прямой ссылки на него и остается видимым для сборщика мусора по средством объекта-финализатора f.

Использование связанного метода отслеживаемого объекта также может препятствовать надлежащему выполнению завершающих операций по освобождению системных ресурсов.

## weakref.proxy

Иногда удобнее использовать прокси, чем слабую ссылку. Прокси-серверы можно использовать, как если бы они были исходным объектом, и их не нужно вызывать до того, как объект станет доступен. Как следствие, они могут быть переданы в библиотеку, которая не знает, что получает ссылку вместо реального объекта

In [ ]:
import weakref


class ExpensiveObject:

    def __init__(self, name):
        self.name = name

    def __del__(self):
        print('(Deleting {})'.format(self))


obj = ExpensiveObject('My Object')
r = weakref.ref(obj)
p = weakref.proxy(obj)

print('via obj:', obj.name)
print('via ref:', r().name)
print('via proxy:', p.name)
del obj
print('via proxy:', p.name)

В случае обращения к прокси-объекгу после удаления объекта, который он
представляет, возбуждается исключение ReferenceError.

## WeakValueDictionary

Класс WeakValueDictionary позволяет создать словарь, в котором значения представлены слабыми ссылками и могут автоматически удаляться сборщиком мусора, если они больше не используются другим кодом. Ниже приведен пример, в котором явные вызовы сборщика мусора используются для демонстрации различий между управлением памятью в случае обычного словаря и словаря WeakValueDictionary.

In [ ]:
import gc
from pprint import pprint
import weakref

gc.set_debug(gc.DEBUG_UNCOLLECTABLE)


class ExpensiveObject:

    def __init__(self, name):
        self.name = name

    def __repr__(self):
        return 'ExpensiveObject({})'.format(self.name)

    def __del__(self):
        print('    (Deleting {})'.format(self))


def demo(cache_factory):
    # Удержать объекты для того, чтобы предотвратить 
    # немедленное удаление слабых ссылок
    
    all_refs = {}
    # Создать кеш
    print('CACHE TYPE:', cache_factory)
    cache = cache_factory()
    for name in ['one', 'two', 'three']:
        o = ExpensiveObject(name)
        cache[name] = o
        all_refs[name] = o
        del o  # уменьшить счетчик ссылок

    print('  all_refs =', end=' ')
    pprint(all_refs)
    print('\n  Before, cache contains:', list(cache.keys()))
    for name, value in cache.items():
        print('    {} = {}'.format(name, value))
        del value  # уменьшить счетчик ссылок

    # Удалить все ссылки на объекты, за исключением тех,
    # которые нахдятся в кеше
    print('\n  Cleanup:')
    del all_refs
    gc.collect()

    print('\n  After, cache contains:', list(cache.keys()))
    for name, value in cache.items():
        print('    {} = {}'.format(name, value))
    print('  demo returning')
    return


demo(dict)
print()

demo(weakref.WeakValueDictionary)

Используемые в циклах for переменные, ссылающиеся на кешируемые значения, должны удаляться явным образом, что и обусловливает необходимость уменьшения счетчиков ссылок на соответствующие объекты. Если этого не сделать, то сборщик мусора не сможет удалить объекты, и они будут оставаться в кеше. Переменная all_refs используется для сохранения ссылок c той целью,чтобы они не были преждевременно удалены сборщиком мусора.